In [25]:
from dotenv import load_dotenv
import os
from notion_utils import *
from openai_chat import *

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
Notion_API_Key = os.getenv("NOTION_TOKEN")
Input_Database_Id = os.getenv("NOTION_DB_INPUT")
Output_Database_Id = os.getenv("NOTION_DB_OUTPUT")

client = OpenAI(api_key=OPENAI_API_KEY)

In [26]:
result = get_database_items_id(Notion_API_Key, Input_Database_Id, 7)
print(result)


予期せぬエラー: list index out of range
Empty DataFrame
Columns: []
Index: []


In [27]:
def get_database_items_id_test(Notion_API_Key, Database_Id, X_Days_Ago=None):


    # ヘッダーを設定
    headers = {
        "Authorization": f"Bearer {Notion_API_Key}",
        "Content-Type": "application/json",
        "Notion-Version": "2022-06-28"
    }

    try:
        # データベースをクエリするリクエスト
        response = requests.post(
            f"https://api.notion.com/v1/databases/{Database_Id}/query",
            headers=headers
        )
        
        # レスポンスを取得
        response.raise_for_status()  # エラーチェック
        data = response.json()
        
        # 各ページの情報を辞書に追加
        results = data.get('results', [])  
        if not results:
            print("データベースにページが存在しません")
            return pd.DataFrame()
        else:
            # 複数のページの情報を格納するためにリストを使用
            pages = []
            
            for result in results:
                
                page_info = {
                    'title': result.get('properties').get('Title').get('title')[0].get('plain_text'),
                    'page_id': result.get('id'),
                    'created_time': result.get('created_time')
                }
                pages.append(page_info)

    except requests.exceptions.RequestException as e:
        print(f"データベースID取得：APIリクエストエラー: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"予期せぬエラー: {e}")
        return pd.DataFrame()

    # DataFrameを作成
    df = pd.DataFrame(pages)

    # 日付でフィルタリング
    if X_Days_Ago is not None:
        current_date = pd.Timestamp.now(tz='UTC')
        x_days_ago = current_date - pd.Timedelta(days=X_Days_Ago)
        df = df[pd.to_datetime(df['created_time']).between(x_days_ago, current_date)]

    return df

get_database_items_id_test(Notion_API_Key, Input_Database_Id, 7)


予期せぬエラー: list index out of range


""


In [28]:
def test_get_database_items_id():
    """get_database_items_id関数のテスト"""
    load_dotenv()
    notion_api_key = os.getenv("NOTION_TOKEN")
    database_id = os.getenv("NOTION_DB_INPUT")
    
    # 基本機能テスト
    df = get_database_items_id(notion_api_key, database_id)
    print("取得したデータフレーム:")
    print(df.head())
    print(f"総行数: {len(df)}")
    
    # 日付フィルタリングテスト
    df_filtered = get_database_items_id(notion_api_key, database_id, X_Days_Ago=7)
    print("\n7日以内のデータ:")
    print(df_filtered.head())
    print(f"フィルタリング後の行数: {len(df_filtered)}")
    
    # エラーケーステスト
    df_error = get_database_items_id("invalid_key", database_id)
    assert df_error.empty, "無効なAPIキーでもDataFrameが返される"
    
    df_error = get_database_items_id(notion_api_key, "invalid_db")
    assert df_error.empty, "無効なデータベースIDでもDataFrameが返される"

if __name__ == "__main__":
    test_get_database_items_id()

予期せぬエラー: list index out of range
取得したデータフレーム:
Empty DataFrame
Columns: []
Index: []
総行数: 0
予期せぬエラー: list index out of range

7日以内のデータ:
Empty DataFrame
Columns: []
Index: []
フィルタリング後の行数: 0
データベースID取得：APIリクエストエラー: 401 Client Error: Unauthorized for url: https://api.notion.com/v1/databases/1e656d7a44ed80abb721ec55373d86e1/query
データベースID取得：APIリクエストエラー: 400 Client Error: Bad Request for url: https://api.notion.com/v1/databases/invalid_db/query
